# Example notebook

In [9]:
import anndata as ad
import numpy as np
import pandas as pd
import pytest
from pydeseq2.utils import load_example_data
from formulaic import model_matrix


@pytest.fixture
def test_counts():
    return load_example_data(
        modality="raw_counts",
        dataset="synthetic",
        debug=False,
    )


@pytest.fixture
def test_metadata():
    return load_example_data(
        modality="metadata",
        dataset="synthetic",
        debug=False,
    )


@pytest.fixture
def test_adata(test_counts, test_metadata):
    return ad.AnnData(X=test_counts, obs=test_metadata)


def test_adata_minimal():
    matrix_format = np.array
    n_obs = 80
    n_donors = n_obs // 4
    rng = np.random.default_rng(9)  # make tests deterministic
    obs = pd.DataFrame(
        {
            "condition": ["A", "B"] * (n_obs // 2),
            "donor": sum(([f"D{i}"] * n_donors for i in range(n_obs // n_donors)), []),
            "other": (["X"] * (n_obs // 4)) + (["Y"] * ((3 * n_obs) // 4)),
            "pairing": sum(([str(i), str(i)] for i in range(n_obs // 2)), []),
            "continuous": [rng.uniform(0, 1) * 4000 for _ in range(n_obs)],
        },
    )
    var = pd.DataFrame(index=["gene1", "gene2"])
    group1 = rng.negative_binomial(20, 0.1, n_obs // 2)  # large mean
    group2 = rng.negative_binomial(5, 0.5, n_obs // 2)  # small mean

    condition_data = np.empty((n_obs,), dtype=group1.dtype)
    condition_data[0::2] = group1
    condition_data[1::2] = group2

    donor_data = np.empty((n_obs,), dtype=group1.dtype)
    donor_data[0:n_donors] = group2[:n_donors]
    donor_data[n_donors : (2 * n_donors)] = group1[n_donors:]

    donor_data[(2 * n_donors) : (3 * n_donors)] = group2[:n_donors]
    donor_data[(3 * n_donors) :] = group1[n_donors:]

    X = matrix_format(np.vstack([condition_data, donor_data]).T)
    return ad.AnnData(X=X, obs=obs, var=var)

In [8]:
adata = test_adata_minimal()

/home/sturm/apps/micromamba/envs/multi-condition-comparisons/lib/python3.10/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [11]:
design = model_matrix("~ condition + C(condition) + donor + continuous", adata.obs)

In [12]:
design.model_spec.term_variables

{1: set(),
 condition: {'condition'},
 C(condition): {'C', 'condition'},
 donor: {'donor'},
 continuous: {'continuous'}}

In [13]:
design.model_spec.variable_terms

{'condition': {C(condition), condition},
 'C': {C(condition)},
 'donor': {donor},
 'continuous': {continuous}}

In [3]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

In [4]:
adata.obs

,SampleName,cell,dex,albut,Run,avgLength,Experiment,Sample,BioSample
SRR1039508,GSM1275862,N61311,untrt,untrt,SRR1039508,126,SRX384345,SRS508568,SAMN02422669
SRR1039509,GSM1275863,N61311,trt,untrt,SRR1039509,126,SRX384346,SRS508567,SAMN02422675
SRR1039512,GSM1275866,N052611,untrt,untrt,SRR1039512,126,SRX384349,SRS508571,SAMN02422678
SRR1039513,GSM1275867,N052611,trt,untrt,SRR1039513,87,SRX384350,SRS508572,SAMN02422670
SRR1039516,GSM1275870,N080611,untrt,untrt,SRR1039516,120,SRX384353,SRS508575,SAMN02422682
SRR1039517,GSM1275871,N080611,trt,untrt,SRR1039517,126,SRX384354,SRS508576,SAMN02422673
SRR1039520,GSM1275874,N061011,untrt,untrt,SRR1039520,101,SRX384357,SRS508579,SAMN02422683
SRR1039521,GSM1275875,N061011,trt,untrt,SRR1039521,98,SRX384358,SRS508580,SAMN02422677


In [5]:
mod = StatsmodelsDE(adata, "~ avgLength + dex")

In [6]:
mod.design

,Intercept,avgLength,dex[T.untrt]
SRR1039508,1.0,126,1
SRR1039509,1.0,126,0
SRR1039512,1.0,126,1
SRR1039513,1.0,87,0
SRR1039516,1.0,120,1
SRR1039517,1.0,126,0
SRR1039520,1.0,101,1
SRR1039521,1.0,98,0


In [7]:
mod.fit()

  0%|          | 0/22810 [00:00<?, ?it/s]

100%|██████████| 22810/22810 [00:39<00:00, 583.60it/s]


### Test a simple contrast

In [8]:
mod.test_contrasts({"treatment_vs_control": mod.contrast("dex", baseline="untrt", group_to_compare="trt")})

  0%|          | 0/22810 [00:00<?, ?it/s]

100%|██████████| 22810/22810 [00:04<00:00, 5456.22it/s]


,pvalue,tvalue,sd,fold_change,contrast
variable,,,,,
ENSG00000161267,7.269989668888602e-07,-30.363177,0.025984,-0.788969,treatment_vs_control
ENSG00000147119,2.1422892379907097e-06,-24.430491,0.043444,-1.061362,treatment_vs_control
ENSG00000134686,3.95000602707298e-06,-21.595136,0.041278,-0.891412,treatment_vs_control
ENSG00000163491,4.217164189577984e-06,21.311721,0.073880,1.574504,treatment_vs_control
ENSG00000113456,5.226221201125236e-06,-20.408056,0.024393,-0.497812,treatment_vs_control
...,...,...,...,...,...
ENSG00000214670,1.0,0.000000,0.000000,0.000000,treatment_vs_control
ENSG00000214643,1.0,0.000000,0.000000,0.000000,treatment_vs_control
ENSG00000214642,1.0,0.000000,0.000000,0.000000,treatment_vs_control


### Build a more complex contrast

In [9]:
adata = sc.datasets.pbmc68k_reduced()
adata.obs

,bulk_labels,n_genes,percent_mito,n_counts,S_score,G2M_score,phase,louvain
index,,,,,,,,
AAAGCCTGGCTAAC-1,CD14+ Monocyte,1003,0.023856,2557.0,-0.119160,-0.816889,G1,1
AAATTCGATGCACA-1,Dendritic,1080,0.027458,2695.0,0.067026,-0.889498,S,1
AACACGTGGTCTTT-1,CD56+ NK,1228,0.016819,3389.0,-0.147977,-0.941749,G1,3
AAGTGCACGTGCTA-1,CD4+/CD25 T Reg,1007,0.011797,2204.0,0.065216,1.469291,G2M,9
ACACGAACGGAGTG-1,Dendritic,1178,0.017277,3878.0,-0.122974,-0.868185,G1,2
...,...,...,...,...,...,...,...,...
TGGCACCTCCAACA-8,Dendritic,1166,0.008840,3733.0,-0.124456,-0.867484,G1,2
TGTGAGTGCTTTAC-8,Dendritic,1014,0.022068,2311.0,-0.298056,-0.649070,G1,1
TGTTACTGGCGATT-8,CD4+/CD25 T Reg,1079,0.012821,3354.0,0.216895,-0.527338,S,0


In [10]:
mod = StatsmodelsDE(adata, "~ n_genes + bulk_labels*phase")
mod.fit()

100%|██████████| 765/765 [00:12<00:00, 62.60it/s] 


In [11]:
mod.test_contrasts(
    {
        "G2M vs S in Dendritic cells": mod.cond(bulk_labels="Dendritic", phase="G2M")
        - mod.cond(bulk_labels="Dendritic", phase="S")
    }
)

100%|██████████| 765/765 [00:02<00:00, 347.07it/s]


,pvalue,tvalue,sd,fold_change,contrast
variable,,,,,
HMGB2,7.379049356876103e-116,28.146733,0.205242,5.776896,G2M vs S in Dendritic cells
STMN1,1.1452526372660046e-35,13.225603,0.282635,3.738015,G2M vs S in Dendritic cells
CALM3,1.9772873146480225e-14,7.824348,0.341087,2.668783,G2M vs S in Dendritic cells
GTF3C6,8.317869063811358e-09,5.835783,0.417569,2.436839,G2M vs S in Dendritic cells
IL32,2.3059965204793834e-08,5.654785,0.296670,1.677604,G2M vs S in Dendritic cells
...,...,...,...,...,...
DTD1,0.9940167079027864,-0.007502,0.398430,-0.002989,G2M vs S in Dendritic cells
IL16,0.9964044894037746,0.004508,0.385664,0.001739,G2M vs S in Dendritic cells
HLA-DQB1,0.9967242650678589,0.004107,0.292488,0.001201,G2M vs S in Dendritic cells


### Test an arbitrary contrast vector 

In [12]:
mod.test_contrasts(
    {
        "G2M vs S in Dendritic cells": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            1.0,
            -1.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            1.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            -1.0,
        ]
    }
)

  0%|          | 0/765 [00:00<?, ?it/s]

100%|██████████| 765/765 [00:00<00:00, 2061.18it/s]


,pvalue,tvalue,sd,fold_change,contrast
variable,,,,,
HMGB2,7.379049356876103e-116,28.146733,0.205242,5.776896,G2M vs S in Dendritic cells
STMN1,1.1452526372660046e-35,13.225603,0.282635,3.738015,G2M vs S in Dendritic cells
CALM3,1.9772873146480225e-14,7.824348,0.341087,2.668783,G2M vs S in Dendritic cells
GTF3C6,8.317869063811358e-09,5.835783,0.417569,2.436839,G2M vs S in Dendritic cells
IL32,2.3059965204793834e-08,5.654785,0.296670,1.677604,G2M vs S in Dendritic cells
...,...,...,...,...,...
DTD1,0.9940167079027864,-0.007502,0.398430,-0.002989,G2M vs S in Dendritic cells
IL16,0.9964044894037746,0.004508,0.385664,0.001739,G2M vs S in Dendritic cells
HLA-DQB1,0.9967242650678589,0.004107,0.292488,0.001201,G2M vs S in Dendritic cells
